<h1>Scraping Fidelity.com</h1>
In this assignment, you will scrape data from fidelity.com. The goal of the exercise is to get the latest sector performance data from the US markets, and to get the total market capitalization for each sector. 

The end result is to write a function: <i>get_us_sector_performance()</i> that will return a list of tuples. Each tuple should correspond to a sector and should contain the following data:
<li>the sector name
<li>the amount the sector has moved
<li>the market capitalization of the sector
<li>the market weight of the sector
<li>a link to the fidelity page for that sector

<p>
The data should be sorted by decreasing order of market weight. I.e., the sector with the highest weight should be in the first tuple, etc.

<h3>Process</h3>
<li>Get a list of sectors and the links to the sector detail pages from the url (see function)
<li>Loop through the list and call the function <i>get_sector_change_and_market_cap(sector_page_link)</i> for each sector
<li>Accumulate the name, the change, the capitalization, the weight and the link for each sector in output_list (see function)
<li>Sort the list by market weight

<b>Notes:</b>
<li>Note that the market weight is a string with a % sign at the back. You will need to get rid of the % and convert the string into a float before you can sort it
<li>Your starting data is the url listed below. You need to extract all data, including links to the sector pages, from the page at this url
<li>To sort a list of tuples by an arbitrary element, use the example at the bottom of this notebook

In [1]:
def get_sector_change_and_market_cap(sector_page_link):
    
    import requests as req
    from bs4 import BeautifulSoup as bs

    response = req.get(sector_page_link)
    results_page = bs(response.content,'lxml')

    data = []
    tags_sector_nums = results_page.find_all('table',class_='snapshot-data-tbl')
    
    for item in tags_sector_nums:
        try:
            sector_change = item.find('td').find('span',class_='negative').get_text()[1:-1]
            sector_change = float(sector_change)
        except:
            sector_change = item.find('td').find('span',class_='positive').get_text()[1:-1]
            sector_change = float(sector_change)
        sector_market_cap = item.find('tbody').find('tr').find_all('td')[1].find('span').get_text()
        sector_market_weight = item.find('td',class_='left-spcng').find('span').get_text()[:-1]        
        if sector_market_weight == '-':
            sector_market_weight = 0.0
        else:
            sector_market_weight = float(sector_market_weight)
        
        
    return sector_change,sector_market_cap,sector_market_weight

In [2]:
def get_us_sector_performance():
    """Returns US Sector Performance"""
    
    import requests as req
    from bs4 import BeautifulSoup as bs
    import itertools as IT

    output = list()
    url = "https://eresearch.fidelity.com/eresearch/goto/markets_sectors/landing.jhtml"
    response = req.get(url)

    results_page = bs(response.content,'lxml')
    tags_sector_name_link = results_page.find_all('a',class_='heading1')
    
    for sector in tags_sector_name_link:
        sector_link = "http://eresearch.fidelity.com" + sector.get('href')
        sector_name = sector.get_text()        
        data_spl_change = get_sector_change_and_market_cap(sector_link)[0]
        data_spl_market = get_sector_change_and_market_cap(sector_link)[1]
        data_spl_weight = get_sector_change_and_market_cap(sector_link)[2]
        
        output.append((sector_name, data_spl_change, data_spl_market,data_spl_weight,sector_link))
    
    output.sort(key=lambda z:z[-2])
    
    return output

get_us_sector_performance()

[('Communication Services',
  0.1,
  '$4.79T',
  0.0,
  'http://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
 ('Materials',
  1.04,
  '$1.96T',
  2.43,
  'http://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=15'),
 ('Real Estate',
  0.85,
  '$1.19T',
  2.65,
  'http://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=60'),
 ('Utilities',
  0.3,
  '$1.22T',
  2.83,
  'http://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=55'),
 ('Energy',
  1.47,
  '$4.07T',
  6.0,
  'http://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=10'),
 ('Consumer Staples',
  0.06,
  '$3.55T',
  6.71,
  'http://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=30'),
 ('Industrials',
  0.9,
  '$4.29T',
  9.7,


In [ ]:
#Test get_sector_change_and_market_cap()
link = "https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25"
get_sector_change_and_market_cap(link)
#Should return (-0.40, $3.58T, 6.80)
#Note that neither the -0.40, nor the 6.80, end with a %sign

In [ ]:
#Test get_us_sector_performance()
get_us_sector_performance()
#Should return (example: obviously the results will vary over time!)
"""
[('Telecommunication Services',
  0.21,
  '$1.74T',
  2.0,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
 ('Materials',
  0.22,
  '$1.95T',
  2.49,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=15'),
 ('Real Estate',
  -0.45,
  '$1.22T',
  2.7,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=60'),
 ('Utilities',
  -0.33,
  '$1.25T',
  2.86,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=55'),
 ('Energy',
  0.76,
  '$3.90T',
  5.83,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=10'),
 ('Consumer Staples',
  -0.32,
  '$3.58T',
  6.8,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=30'),
 ('Industrials',
  0.72,
  '$4.31T',
  9.83,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=20'),
 ('Consumer Discretionary',
  1.03,
  '$5.76T',
  12.9,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
 ('Financials',
  0.39,
  '$7.45T',
  13.71,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'),
 ('Health Care',
  0.76,
  '$5.70T',
  14.71,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
 ('Information Technology',
  0.81,
  '$9.84T',
  26.17,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45')]
  
"""

<h3>Sorting</h3>

In [ ]:
x = [('a',23.2,'b'),('c',17.4,'f'),('d',29.2,'z'),('e',1.74,'bb')]
x.sort() #Sorts by the first element of the tuple
x

In [ ]:
x = [('a',23.2,'b'),('c',17.4,'f'),('d',29.2,'z'),('e',1.74,'bb')]
x.sort(key=lambda k: k[1]) #Sorts by the element at position 1
x